In [ ]:
import pandas as pd
import re
import toml
import pygsheets
import requests
import Bio.Restriction as Restriction
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.workflow as workflow
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.api.geneious as geneious

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"])

# Config

In [ ]:
olib_oligos = reg[("oLIB", "oligos")]
olt_oligos = reg[("oLT", "oligos")]
plib_plasmids = reg[("pLIB", "plasmids")]
plib_maps = reg[("pLIB", "maps")]
flib_fragments = reg[("fLIB", "fragments")]
part_types = reg[("fLIB", "fragments", "Part types")]

# Digest test

In [ ]:
len(reg.get("fLIB255")["_seq"])

In [ ]:
s = reg.get("mKate2_nocut")
s

In [ ]:
len(s["_seq"])

In [ ]:
s["_seq"]

In [ ]:
s = reg.get("pLIB112")["_seq"]

In [ ]:
reg.eval_expr("pLIB112/AarI/BbsI")

# GG test

In [ ]:
import networkx as nx
import random

In [ ]:
import paulssonlab.cloning.design as design

In [ ]:
design.golden_gate_placeholder?

In [ ]:
def _make_frag(overhang1, overhang2):
    return enzyme.re_digest(
        design.golden_gate_placeholder(
            Restriction.BsaI, Restriction.BbsI, overhang1, overhang2
        ),
        Restriction.BbsI,
    )[1]

In [ ]:
test_set = [
    _make_frag("aaaa", "gggg"),
    _make_frag("gggg", "aatt"),
    _make_frag("aatt", "ggcc"),
    _make_frag("ggcc", "gggg"),
    _make_frag("aaaa", "cccc"),
    #_make_frag("gggg", "cccc"),
]
random.shuffle(test_set)

In [ ]:
print(_make_frag("aaaa", "gggg"))

In [ ]:
sequence._assemble_goldengate(test_set[0], test_set[1])

In [ ]:
sequence._assemble_goldengate(test_set[0], test_set[2])

In [ ]:
graph = nx.DiGraph()
for idx1 in range(len(test_set)):
    for idx2 in range(len(test_set)):
        score = sequence._assemble_goldengate(test_set[idx1], test_set[idx2])[1]
        if score > 0:
            graph.add_edge(idx1, idx2)
cycles = list(nx.simple_cycles(graph))
if len(cycles) == 0:
    raise ValueError("")
elif len(cycles) > 1:
    raise ValueError("")
cycle = cycles[0]
seq = test_set[cycle[0]]
for idx in cycle[1:]:
    new_seq, score = sequence._assemble_goldengate(seq, test_set[idx])
    assert score > 0
    seq = new_seq

In [ ]:
nx.draw(graph, with_labels=True)

In [ ]:
test_set

In [ ]:
sequence.assemble_circular(test_set)

In [ ]:
g = sequence.assembly_graph(test_set, method="goldengate")
g.edges

In [ ]:
g

In [ ]:
nx.draw(g, with_labels=True)

# Barcode GFA

In [ ]:
" + ".join(
    [
        "barcode_bit{num}_on | barcode_bit{num}_on".format(num=num)
        for num in range(1, 31)
    ]
)

In [ ]:
x = reg.get("barcode_bit1_on")

In [ ]:
x["_seq"]

In [ ]:
### LOOK AT GIBSON/GG

# desiderata:
# drop-in replacement for SeqRecord/DsSeqRecord (?)
# fasta/gb export
# circular/noncircular
# for GG/gibson: hhh

# from_gfa
# to_gfa

# __or__ (|) [also implement for DsSeqRecord]
# max_length, min_length
# __len__ raises error or returns None if lengths not equal?
# slicing
# __add__, __radd__
# ligation check
# overhangs


class VariableSeqRecord:
    pass


class VariableSeq:
    def __init__(self, seqs):
        pass

    def from_gfa():
        pass

    def to_gfa():
        pass

    def squeeze():
        pass